<a href="https://colab.research.google.com/github/synkodev/gemini-alura-challenge/blob/main/gemini_alura_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Using Google's Generative AI to recommend dishes based on user input

## Apresentação do projeto

👋 Seja bem-vindo ao meu projeto realizado com os conhecimentos adquiridos na semana de Imersão IA Alura + Google. A intenção deste projeto é de colocar em prática os conhecimentos adquiridos durante essa semana de imersão, na qual aprendemos o que é o Gemini, como efetivamente aplicar técnicas eficazes de prompting, o básico da utilização da SDK do Python e os conceitos por trás da IA Generativa.

📘 O projeto consiste em utilizar a IA para fazer recomendações de pratos e responder outras questões do usuário com base em menus disponíveis e pré-determinados. Isso poderia ser muito útil considerando aplicativos de entrega de comida, por exemplo, no qual o usuário poderia selecionar alguns restaurantes e fazer perguntas com base nas opções fornecidas pelos mesmos. Para isso, utilizarei o modelo de embedding para "varrer" os menus informados, encontrando o trecho do documento que mais se adequa à busca do usuário. Na sequência, utilizarei o modelo Generativo para gerar uma resposta adequada ao usuário. Para este fim, utilizarei o Gemini 1.5 Pro.

Espero que você goste deste projeto e me diga o que você achou no meu [LinkedIn](https://www.linkedin.com/in/walencar) ou no meu [GitHub](https://www.github.com/synkodev). Obrigado e boa leitura! 😊
<br>
<br>
<hr>

## English introduction

👋 Welcome to my final project of the IA Week Alura + Google event. This project aims to practice all the knowledge I've acquired during this intensive learning week, in which we were told what is Gemini, how to correctly apply efficient prompting techniques, the basics of Python's SDK for generative AI and so on.

📘 The project consists of using AI to recommend dishes or answer any inquiries from the user based on pre-determined available menus. This could be useful when applied in the context of delivery apps, where the user could possibly select a few restaurants of their choice and then ask a few questions to find a dish of their interest without having to search each menu individually. To accomplish this, I'll use the embedding model to search in the menus passages that are most relevant according to the user's input or inquiry. Once that is done, I'll use the generative AI model to generate an adequare response to the user. For this, Gemini 1.5 Pro will be used.

I hope you like this project! Most texts will be in Portuguese, but the code commentary will be in English at most times. Tell me what you think about this project on my [LinkedIn](https://www.linkedin.com/in/walencar) profile or you cand find me on [GitHub](https://www.github.com/synkodev) as well. Thank you for reading! 😊

## Pré-requisitos de execução

Primeiramente, precisamos instalar a SDK para Python da IA do Google. O Colab já possui algumas bibliotecas instaladas no ambiente por padrão, e que portanto não precisam ser *instaladas*, apenas mencionadas no passo de importação (a seguir).

In [1]:
!pip install -q -U google-generativeai # Install the Python SDK

Uma vez instaladas todas as nossas dependências, podemos prosseguir com a importação da biblioteca recém-instalada e das demais que necessitaremos para trabalhar com nossos dados. Nomearemos um *alias* que facilite a chamada dentro do código como **genai**.

*   `textwrap`: para formatação de texto.
*   `numpy`: para operações matemáticas avançadas.
*   `pandas`: para utilizarmos a estrutura de DataFrame.
*   `google.generativeai`: nossa IA do Google
*   `userdata`: para "puxarmos" nossa chave de API do cofre do Colab.



In [2]:
import textwrap # Text formatting

import numpy as np # Numpy for math operations
import pandas as pd # Pandas for exploring DataFrames

import google.generativeai as genai # Imports the library to work with AI
from google.colab import userdata # Allows to keep secret keys @ Colab Secrets

## Primeiros passos
Começaremos utilizando nossa chave de API do AI Studio atravéis da Secret salva no Colab. A chave da API é necessária para configurar a inicialização do modelo, que mais tarde instanciaremos através do método `genai.GenerativeModel`.

Uma vez inserida nossa chave, pediremos uma lista de modelos disponíveis que sejam de geração de conteúdo e *embedding*. Utilizaremos o nome dos modelos ao inicializar um novo modelo.

In [3]:
# Gets the secret key kept @ Colab
api_key = userdata.get("gemini")
genai.configure(api_key=api_key)

# Iteraction to show every name for embedding and generative models
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods or "embedContent" in m.supported_generation_methods:
    print(m.name);

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004


Perfeito! Com base nos nomes disponíveis acima, separaremos em duas variáveis quais modelos utilizaremos para a geração da resposta no final do projeto e para varrer o documento em busca de trechos que tenham maior relação com a pergunta feita pelo usuário.

Por hora, criaremos um modelo generativo utilizando o **Gemini 1.5 Pro**, apontando para a última versão disponível para este modelo.

In [4]:
# Create variable with strings for each model we'll use
generative_model = "gemini-1.5-pro-latest"
embedding_model = "models/embedding-001"

# Instantiate a new generative model pointing to the Gemini 1.5 Pro, later used for prompting
model = genai.GenerativeModel(model_name=generative_model)

## Criando os dados utilizados

A seguir, informaremos quais os documentos que serão levados em consideração para realizar o embedding. Isto é, dado o input do usuário, o modelo irá passar pelo conteúdo destes três documentos que escreveremos abaixo, simulando menu de diferentes restaurantes, e encontrará qual o menu que atende melhor ao que foi perguntando pelo usuário.

Para a criação dos documentos utilizaremos duas chaves. `"Restaurant"` armazenará o nome do restaurante, enquanto `"Menu"` salvará todas as informações relacionadas aos pratos, como nome, preço, e os ingredientes que cada um contém.

Por fim, salvaremos os três documentos, todos com o mesmo formato de chave-valor, na variável `documents` que será usada para realizar o *embedding*.

In [5]:
# Creating three documents for embedding
DOCUMENT1 = {
    # First key: Restaurant, stores the restaurant name
    "Restaurant": "Chef's Kiss",
    # Second key: Menu, stores all info regarding the dish name, price and ingredients, separated by this '---' structure
    "Menu": """
    Salmon Crab Cakes

    13,90 euros

    fresh blue crab
    salmon
    sriracha
    curried mango
    pineapple chutney
    ---
    Boneless Beef Short Rib

    12 euros

    piece of rib
    beer
    cherries
    onoins
    parnsip puree
    spaetzle
    seasonal vegetables
    ---
    Chicken Supreme

    9,90 euros

    chicken
    wild mushrooms
    caramelized onions
    garlic whipped potatoes
    seasonal vegetables
    """
}

DOCUMENT2 = {
    "Restaurant": "Veggie Maggie",
    "Menu": """
    BUTTERNUT SQUASH MALAYSIAN CURRY

    15,95 euros

    jackfruit
    pumpkin
    mushrooms
    turmeric
    curry sauce
    brown basmati
    rice
    peanuts
    coriander
    ---
    RAW VEGAN LASAGNA

    14,35 euros

    raw zucchini
    fresh tomatoes and dried tomatoes sauce
    Goji berries
    cashews and macadamia nuts 'cheese-like' cream
    pico de gallo
    ---
    PLANT-BASED TRUFFLE MAYO BURGER

    12,95 euros

    whole spelt gluten-free brioche bread
    plant-based hamburger
    truffled mayonnaise
    sautéed mushrooms
    roasted onion
    plant-based 'cheddar'
    roasted sweet potatoes with plant-based yogurt sauce
    """
}

DOCUMENT3 = {
    "Restaurant": "McClown's",
    "Menu": """
    Big Clown

    7 euros

    90g Hamburguer
    Lettuce
    Tomatoes
    Mayonnaise
    Picles
    Cheddar
    ---
    Fish Clown Burguer

    6 euros

    Fish hamburguer
    Lettuce
    Tomatoes
    Mayonnaise
    Cheddar
    Lemon-based sauce dip
    """
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

Para facilitar a manipulação dos documentos, converteremos esses documentos todos em uma estrutura de dataframe.

In [6]:
# Converting the documents created above to a DF structure to further manipulate it easily
df = pd.DataFrame(documents)
df.columns = ["Restaurant", "Menu"]
print(df)

      Restaurant                                               Menu
0    Chef's Kiss  \n    Salmon Crab Cakes\n\n    13,90 euros\n\n...
1  Veggie Maggie  \n    BUTTERNUT SQUASH MALAYSIAN CURRY\n\n    ...
2      McClown's  \n    Big Clown\n\n    7 euros\n\n    90g Hamb...


Acima, podemos ver a representação dos dados contidos na chave `"Restaurant"` na primeira coluna, e os dados da chave `"Menu"` na segunda. Além disso, cada documento que criamos se tornou uma linha (row) do DataFrame `df`.

## Embedding

Agora, para realizarmos a nossa busca com base no input do usuário e facilitar a aproximação semântica entre cada documento, vamos transformar o conteúdo de cada documento em algo que seja mais próximo do que o nosso algoritmo efetivamente entende. E é justamente isso que é o *embedding*: a conversão desses textos em uma estrutura de vetores numéricos que pode ser processada por algoritmos de ML, e essa estrutura é projetada **para capturar o valor semântico e o contexto que cada "palavra" representa**.

Precisamos gerar esse vetor para cada item do nosso DataFrame.

In [7]:
# This function will be applied to each row of `df` and create a new column containing the embedded value of each row
# Embedding will turn all the text content in a numerical vector representation, projected to identify context and semantical value
def embed_fn(restaurant, menu):
  return genai.embed_content(model=embedding_model,
                             content=menu,
                             task_type="retrieval_document",
                             title=restaurant)["embedding"]

Executando a função acima para cada linha de `df`, geramos uma coluna `"Embeddings"` que contém a sua representação vetorizada.

In [8]:
# Creating 'Embeddings' column by applying `embed_fn`
df['Embeddings'] = df.apply(lambda row: embed_fn(row['Restaurant'], row['Menu']), axis=1)
print(df)

      Restaurant                                               Menu  \
0    Chef's Kiss  \n    Salmon Crab Cakes\n\n    13,90 euros\n\n...   
1  Veggie Maggie  \n    BUTTERNUT SQUASH MALAYSIAN CURRY\n\n    ...   
2      McClown's  \n    Big Clown\n\n    7 euros\n\n    90g Hamb...   

                                          Embeddings  
0  [0.050908875, -0.009233177, 0.002469926, -0.04...  
1  [0.003577931, -0.042687457, 0.00065681455, -0....  
2  [0.0123029705, -0.043154325, -0.030352496, -0....  


## Recebendo input de usuário

Agora, pediremos um input de usuário. Pode ser uma pergunta, do tipo "Qual o restaurante com mais opções veganas?" ou ainda uma ordem do tipo "Sugira um prato com alto teor calórico".

In [16]:
# Requests for user input
user_input = input("Enter question or prompt: ")

Enter question or prompt: lalaland


Assim como fizemos com os dados do nosso documento, também precisamos transformar o que o nosso usuário nos passou para uma estrutura de dados que seja mais familiar para o algoritmo de aprendizado de máquina. Vamos gerar o embedding desse input de usuário.

In [10]:
# Also turns the user input into embedded information, so it can be compared to the semantic value of each row of our `df`
request = genai.embed_content(model=embedding_model,
                              content=user_input,
                              task_type="retrieval_query")
print(request)

{'embedding': [0.019074757, -0.056308046, -0.029934365, 0.041971743, 0.027336726, -0.0061310343, 0.014299876, -0.015082191, 0.024205603, 0.04964768, 0.015723864, -0.013006708, -0.026740521, -0.040987577, -0.0068481066, 0.016006256, -0.032048646, -0.0034223504, 0.030954586, -0.045143455, 0.0037859464, -0.000709151, -0.03530885, -0.022025129, -0.0067549367, 0.005427041, 0.0067314343, -0.047103796, -0.01379762, 0.0134636145, -0.09312128, 0.06791194, -0.0718045, -0.022353198, -0.023214562, -0.08256519, 0.0011906382, 0.052671757, 0.035843957, 0.060302105, 0.0072748912, -0.007934607, -0.05497862, -0.0041299444, 0.019347496, -0.07355151, -0.025041033, 0.010869258, -0.01641778, -0.07251568, -0.008359695, -0.026266156, 0.05305042, 0.0007017568, -0.032303378, -0.088730074, 0.04033678, 0.03214192, -0.022957103, 0.036769334, -0.017328735, 0.026666777, -0.010661408, 0.029052706, 0.040886257, -0.016199635, -0.0008797388, -0.0074425056, 0.045768443, 0.0032213451, 0.0694455, -0.057440437, 0.012748921,

## Buscando o documento
Com isso, conseguimos fazer uma busca com o valor semântico do input de usuário e o valor semântico de cada documento, a partir da qual o modelo conseguirá entender qual é a opção que mais se adequa à busca feita pelo nosso usuário. Para isso contaremos com a ajuda do `numpy`.

In [11]:
def find_best_passage(user_input, dataframe):
  query_embedding = genai.embed_content(model=embedding_model,
                                        content=user_input,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Menu'] # Return text from index with max value

Uma vez encontrado o trecho que possui maior aproximação com o que foi solicitado pelo usuário, retornamos o menu que tem maior pertinência com a busca.

In [12]:
result_text = find_best_passage(user_input, df)
print(result_text)


    Big Clown

    7 euros

    90g Hamburguer
    Lettuce
    Tomatoes
    Mayonnaise
    Picles
    Cheddar
    ---
    Fish Clown Burguer

    6 euros

    Fish hamburguer
    Lettuce
    Tomatoes
    Mayonnaise
    Cheddar
    Lemon-based sauce dip
    


## Transformando em texto

Por fim, agora podemos transformar o resultado dessa busca vetorizada em algo mais humanizado. A vantagem de realizar essa busca a partir do embedding é que garantimos que a fonte da nossa informação virá somente dos dados que fornecemos ao modelo, evitando assim até certo ponto alucinações provenientes do excesso de informações disponíveis para a IA Generativa.

Agora, vamos transformar esse resultado em uma resposta mais amigável para o usuário. Para isso, utilizaremos mais uma vez nosso modelo generativo, o Gemini 1.5 Pro.

Criaremos uma função contendo um prompt, com base nos parâmetros recebidos: o input do usuário e o texto mais relevante de acordo com a busca feita anteriormente usando *embedding*. Pediremos ao modelo que:

*   Seja sucinto e foque no que foi perguntado de maneira precisa
*   Seja o mais direto e conciso possível
*   Use dados externos somente para trazer informações nutricionais (que não estão contidadas nos documentos fornecidos), possíveis riscos a saúdo e restrições alimentares relacionadas.
*   Se a questão estiver fora do contexto, sinalize educadamente que não pode responder a isso.
*   Se a questão estiver dentro do contexto mas sem resultados relevantes o suficiente, informe que não obteve resultados suficientemente significativos.


In [17]:
# Creates the prompt that will be passed to the generative model, in order to create a more friendly and custom response to the user
# Returns the prompt, containing the instructions (i.e.: tone), alternative scenarios, the question and the result text from the embedding
def create_formatted_prompt(user_input, result_text):
  escaped_text = result_text.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You're intended to synthetize the result text below, adapting the answer to what has been specifically asked. \
  Make sure the answer is as direct and concise as possible. \
  You can only bring data that is not directly mentioned in the result text if you use it to complement small pieces of information, keeping it short and concise. \
  Scenarios where you can use external data to give an answer: gathering more information about an ingredient (nutrition facts),
  potential benefits or harm of each ingredient for human health, diet restrictions related to any ingredient. \
  The answer must be given in the same language as the input question, and mandatorily must have to do with the result text. \
  If the question is out of the context and has anything to do with the result text, apologize and inform you can't answer the question.
  If the question is in the context but still has no relevant results, apologize and inform that the question had no relevant results given the presented result text.
  QUESTION: '{user_input}'
  RESULT TEXT: '{escaped_text}'
    ANSWER:
  """).format(user_input=user_input, escaped_text=escaped_text)
  return prompt

Podemos ver abaixo o prompt pronto:

In [14]:
prompt = create_formatted_prompt(user_input, result_text)
print(prompt)

You're intended to synthetize the result text below, adapting the answer to what has been specifically asked.   Make sure the answer is as direct and concise as possible.   You can only bring data that is not directly mentioned in the result text if you use it to complement small pieces of information, keeping it short and concise.   Scenarios where you can use external data to give an answer: gathering more information about an ingredient (nutrition facts),
  potential benefits or harm of each ingredient for human health, diet restrictions related to any ingredient.   The answer must be given in the same language as the input question.   If the question is out of the context, apologize and inform you can't answer the question.
  If the question is in the context but still has no relevant results, apologize and inform that the question had no relevant results given the presented result text. 
  QUESTION: 'Sugira um prato com alto teor calórico'
  RESULT TEXT: '     Big Clown      7 eur

E por, fim, a resposta do modelo para usuário:

In [15]:
answer = model.generate_content(prompt)
print(answer.text)

O Big Clown tem alto teor calórico por conter hambúrguer, maionese e cheddar.  



# Conclusão

O projeto passa com êxito pelas principais funcionalidades do Gemini. Para a tarefa escolhida, não julguei necessário configurações mais avançadas, como `temperature`, `top_K`, `top_P` e por isso segui como Gemini 1.5 Pro, utilizando seus valores padrão. Em um projeto em que a mudança desses valores fosse necessária, seria recomendada a utilização do Gemini 1.0 Pro, que é um modelo já lapidado que possui essas parametrizações disponíveis.

Concluir este projeto me permitiu identificar aplicações práticas e muito úteis da AI em situações reais no mundo em que vivemos hoje. Agradeço à Alura e à Google pelos conhecimentos fornecidos durante a imersão que me possibilitaram a construção deste pequeno projeto, que possui grande potencial de aplicação em produtos que usamos todos os dias para pedir comida, e que possui requisitos técnicos que poderei utilizar em projetos futuros.